In [1]:
from dataDownloader  import DataDownloader
from db_financialStatement import DB_FinancialStatement
from db_nyse import DB_NYSE
from db_stock import DB_Stock
from portfolio import Portfolio
from assetAllocation import AssetAllocation

import datetime
import dataDownloader
import yfinance as yf;
import pandas as pd
import commonHelper
import numpy as np


In [ ]:
from matplotlib.dates import relativedelta
from datetime import datetime, timedelta

from commonHelper import EDateType

df_rank = pd.read_csv('market_cap_rank.csv')
# df_rank = df_rank.head(20)

symbol_list = []
for col in df_rank.columns:
    s = df_rank[col].tolist()
    symbol_list.append(s)

symbol_list = [x for row in symbol_list 
                    for x in row]
symbol_list = [x for x in symbol_list 
                    if pd.notna(x)]

quarter_list = df_rank.columns.tolist()
# date_dict = commonHelper.get_date_dict_by_quarter_lazy(quarter_list)
# oldest, latest = commonHelper.get_date_range_from_quarters(date_dict)

start_date = '2013-12-01'
end_date = '2025-08-04'

date_dict = commonHelper.get_date_dict_by_quarter_lazy(quarter_list)
date_dict = commonHelper.get_trimmed_date_dict(date_dict, start_date, end_date)
# date_dict = commonHelper.adjust_start_data_dict_by_quarter(date_dict, quarter_list[0])
oldest, latest = commonHelper.get_date_range_from_quarters(date_dict)

dt_oldest = datetime.strptime(oldest, "%Y-%m-%d")
dt_oldest = dt_oldest - relativedelta(years=1)
oldest = dt_oldest.strftime("%Y-%m-%d")

symbols_dfs = AssetAllocation.get_stock_data_with_ma(symbol_list, oldest, latest, [10], 'ma_month', True)
symbols_dfs = AssetAllocation.filter_close_last_month(symbols_dfs)


def relative_momentum(symbols_dfs:dict, df_rank:pd.DataFrame):

    for symbol, df in symbols_dfs.items():
        df['Result Close Before 12M'] = df['Result Close'].shift(12)
        df['Return Ratio 12M'] = (df['Result Close']/df['Result Close Before 12M']) - 1
        df = df.dropna(subset=['Return Ratio 12M'])
        symbols_dfs[symbol] = df

    df = AssetAllocation.merge_to_dfs(symbols_dfs)

    columns = [col for col in df.columns if col not in ['Adj Close','Close','Dividends','Condition', 'Result Close Before 12M']]
    df = df[columns]


    def get_quarter(row):
        date = row['Date']
        quarter = commonHelper.get_quarter_by_date(date)
        return quarter

    def get_selected_assets(row, top_n=20):
        date = row['Date']
        symbols = row['Symbol']
        ratios = row['Return Ratio 12M']

        ratios_dict = {}
        for idx in range(len(symbols)):
            ratios_dict[symbols[idx]] = ratios[idx]

        quarter = commonHelper.get_quarter_by_date(date)
        
        rank_symbols = df_rank[quarter].tolist()

        sorted_symbols = sorted(
            rank_symbols,
            key=lambda sym: ratios_dict.get(sym, float('-inf')),
            reverse=True
        )

        sorted_symbols = sorted_symbols[:top_n]
        sorted_symbols.sort()
        return sorted_symbols
    

    
    def get_restart_return_ratio_12M(row):
        restart_asset = row['Restart Asset']
        symbol_list = row['Symbol']
        ratio_12M_list = row['Return Ratio 12M']
        symbol_to_ratio_12M = dict(zip(symbol_list, ratio_12M_list))
        restart_ratio_12M_list = [
            symbol_to_ratio_12M.get(symbol, 0) 
            for symbol in restart_asset
        ]
        return restart_ratio_12M_list

    df['End Balance'] = None
    df['Balance'] = None
    df['Quarter'] = df.apply(get_quarter, axis=1)
    df['Restart Asset'] = df.apply(get_selected_assets, axis=1, top_n=10)
    
    # 분기별 리벨런싱을 위한 리스타트 에셋 재설정 (이거 없으면, 매월 리벨런싱이 된다)
    for i in range(1, len(df)):
        prev_quarter = df.at[i-1, 'Quarter']
        quarter = df.at[i, 'Quarter']
        if prev_quarter == quarter:
            df.at[i,'Restart Asset'] = df.at[i-1, 'Restart Asset']
    
    df['Restart Return Ratio 12M'] = df.apply(get_restart_return_ratio_12M, axis=1)
    df['Restart Balance'] = None
    df['Do Rebalance'] = False
    df = df.drop(columns='Return Ratio 12M')

    def get_restart_result_close(prev_symbol: list, prev_close:list , prev_restart_asset:list) -> list:
        symbol_to_close = dict(zip(prev_symbol, prev_close))
        restart_result_close = [symbol_to_close.get(sym, 0) for sym in prev_restart_asset]
        return restart_result_close

    for i in range(len(df)):
        restart_asset = df.at[i, 'Restart Asset']
        restart_len = len(restart_asset)
        if i == 0:
            init_balance = 10000
            df.at[i, 'Balance'] = init_balance
            df.at[i, 'Restart Balance'] = [init_balance//(len(restart_asset)) for x in range(restart_len)]
        
        else:
            prev_restart_assets = df.at[i-1, 'Restart Asset']
            prev_restart_result_close = get_restart_result_close(df.at[i-1, 'Symbol'], df.at[i-1, 'Result Close'], prev_restart_assets)
            curr_restart_result_close = get_restart_result_close(df.at[i, 'Symbol'], df.at[i, 'Result Close'], prev_restart_assets)
            
            change_factors = [
                curr_restart_result_close[j] / prev_restart_result_close[j]
                if prev_restart_result_close[j] != 0 else 0
                for j in range(len(prev_restart_result_close))
            ]

            end_balances = [round(df.at[i-1, 'Restart Balance'][j] * change_factors[j], 2) for j in range(len(change_factors))]
            df.at[i, 'End Balance'] = end_balances

            total_balance = sum(end_balances)
            df.at[i, 'Balance'] = total_balance

            curr_restart_assets = df.at[i, 'Restart Asset']

            if prev_restart_assets == curr_restart_assets:
                df.at[i, 'Restart Balance'] = end_balances
            else:
                df.at[i, 'Restart Balance'] = [total_balance//restart_len] * restart_len
                df.at[i, 'Do Rebalance'] = True
            

    return df


relative_momentum(symbols_dfs, df_rank)



,Date,Symbol,Result Close,MMA_10,End Balance,Balance,Quarter,Restart Asset,Restart Return Ratio 12M,Restart Balance,Do Rebalance
0,2023-11-30,"[ADN, AGAE, AGEN, AMSC, AQB, ARAY, ARTW, BKYI,...","[11.16, 0.91, 15.6, 9.46, 1.71, 2.61, 2.06, 3....","[22.63, 1.04, 27.59, 6.79, 7.47, 3.18, 2.33, 1...",None,10000,2023-Q3,"[AGAE, CLIR, EDRY, FPAY, HHS, ICCC, IZEA, MOGO...","[-0.35, 0.8, 0.23, 0.33, -0.32, -0.34, -0.09, ...","[1000, 1000, 1000, 1000, 1000, 1000, 1000, 100...",False
1,2023-12-29,"[ADN, AGAE, AGEN, AMSC, AQB, ARAY, ARTW, BKYI,...","[6.69, 1.06, 16.6, 11.14, 2.85, 2.83, 2.07, 3....","[18.05, 1.0, 25.2, 7.37, 6.44, 3.18, 2.33, 9.4...","[1164.84, 895.16, 962.12, 1159.72, 997.06, 112...",10739.74,2023-Q3,"[AGAE, CLIR, EDRY, FPAY, HHS, ICCC, IZEA, MOGO...","[0.01, 1.06, 0.11, 0.8, -0.42, -0.17, -0.07, 0...","[1164.84, 895.16, 962.12, 1159.72, 997.06, 112...",False
2,2024-01-31,"[ADN, AGAE, AGEN, AMSC, AQB, ARAY, ARTW, BKYI,...","[4.83, 1.18, 13.4, 11.0, 2.53, 2.59, 2.05, 2.0...","[15.35, 1.0, 23.56, 7.97, 5.5, 3.14, 2.33, 8.3...","[1296.71, 798.39, 1060.1, 923.61, 1136.56, 116...",10506.26,2023-Q3,"[AGAE, CLIR, EDRY, FPAY, HHS, ICCC, IZEA, MOGO...","[-0.03, 0.34, 0.32, 0.17, -0.38, -0.28, -0.3, ...","[1296.71, 798.39, 1060.1, 923.61, 1136.56, 116...",False
3,2024-02-29,"[ADN, AGAE, AGEN, AMSC, AQB, ARAY, ARTW, BKYI,...","[6.3, 0.95, 13.4, 13.45, 2.4, 2.6, 1.97, 2.1, ...","[13.7, 0.99, 21.88, 8.92, 4.66, 3.07, 2.26, 7....","[1043.96, 1032.26, 1111.11, 847.22, 1129.22, 1...",10916.4,2023-Q3,"[AGAE, CLIR, EDRY, FPAY, HHS, ICCC, IZEA, MOGO...","[-0.32, 0.58, 0.18, 0.17, -0.28, -0.11, -0.23,...","[1043.96, 1032.26, 1111.11, 847.22, 1129.22, 1...",False
4,2024-03-28,"[ADN, AGAE, AGEN, AMSC, AQB, ARAY, ARTW, BKYI,...","[5.4, 1.01, 11.6, 13.51, 2.02, 2.47, 2.1, 2.1,...","[11.78, 0.99, 19.92, 9.8, 3.89, 2.94, 2.23, 6....","[1109.89, 782.26, 1052.52, 1034.72, 1098.38, 1...",11158.21,2023-Q4,"[AMSC, ARAY, CCEC, GENC, GP, GPRK, HVT, ITRN, ...","[1.75, -0.17, 0.3, 0.09, -0.15, -0.15, 0.07, 0...","[1115.0, 1115.0, 1115.0, 1115.0, 1115.0, 1115....",True
5,2024-04-30,"[ADN, AGAE, AGEN, AMSC, AQB, ARAY, ARTW, BKYI,...","[3.75, 0.85, 12.12, 12.35, 1.92, 2.13, 1.89, 1...","[10.39, 0.97, 17.93, 10.41, 3.37, 2.77, 2.18, ...","[1019.26, 961.52, 1071.43, 1133.71, 1052.74, 1...",10488.01,2023-Q4,"[AMSC, ARAY, CCEC, GENC, GP, GPRK, HVT, ITRN, ...","[2.05, -0.36, 0.32, 0.29, -0.34, -0.15, 0.02, ...","[1019.26, 961.52, 1071.43, 1133.71, 1052.74, 1...",False
6,2024-05-31,"[ADN, AGAE, AGEN, AMSC, AQB, ARAY, ARTW, BKYI,...","[3.59, 0.69, 15.69, 19.18, 1.75, 1.77, 1.74, 1...","[8.11, 0.94, 16.46, 11.32, 2.77, 2.52, 2.09, 3...","[1582.95, 799.01, 1145.76, 1294.05, 667.87, 12...",10845.91,2024-Q1,"[BKYI, FTFT, INM, IQST, NBY, RKDA, SBET, SGBX,...","[-0.86, -0.35, -0.78, 1.94, -0.85, -0.33, -0.7...","[1084.0, 1084.0, 1084.0, 1084.0, 1084.0, 1084....",True
7,2024-06-28,"[ADN, AGAE, AGEN, AMSC, AQB, ARAY, ARTW, BKYI,...","[3.05, 1.27, 16.75, 23.39, 1.61, 1.82, 1.56, 1...","[6.73, 0.97, 15.38, 12.73, 2.38, 2.42, 1.98, 3...","[916.37, 650.4, 977.44, 1039.0, 696.41, 1115.8...",9242.65,2024-Q1,"[BKYI, FTFT, INM, IQST, NBY, RKDA, SBET, SGBX,...","[-0.88, -0.63, -0.71, 0.91, -0.92, -0.19, -0.8...","[916.37, 650.4, 977.44, 1039.0, 696.41, 1115.8...",False
8,2024-07-31,"[ADN, AGAE, AGEN, AMSC, AQB, ARAY, ARTW, BKYI,...","[4.54, 1.53, 6.04, 24.16, 1.64, 1.84, 1.44, 1....","[6.01, 1.03, 13.72, 14.39, 2.07, 2.33, 1.89, 2...","[810.21, 528.45, 646.73, 899.92, 189.07, 981.2...",7599.39,2024-Q1,"[BKYI, FTFT, INM, IQST, NBY, RKDA, SBET, SGBX,...","[-0.88, -0.72, -0.84, -0.33, -0.98, -0.36, -0....","[810.21, 528.45, 646.73, 899.92, 189.07, 981.2...",False
9,2024-08-30,"[ADN, AGAE, AGEN, AMSC, AQB, ARAY, ARTW, BKYI,...","[3.0, 1.27, 5.15, 20.22, 1.02, 2.18, 2.07, 1.3...","[5.23, 1.07, 12.64, 15.79, 1.95, 2.28, 1.9, 2....","[743.16, 474.25, 1304.48, 736.3, 154.41, 1101....",8205.57,2024-Q2,"[BOXL, BTCT, EVGN, FORD, IMCC, NTRP, POAI, RKD...","[-0.81, -0.26, -0.56, -0.55, -0.55, 

In [3]:
df = yf.download(['VRME'], '2025-06-01', '2025-07-01')
display(df)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,VRME,VRME,VRME,VRME,VRME
Date,,,,,
2025-06-02,0.7220,0.7399,0.7100,0.7200,50200
2025-06-03,0.7499,0.7500,0.7250,0.7300,42400
2025-06-04,0.8054,0.8186,0.7315,0.7500,98900
2025-06-05,0.7950,0.8100,0.7800,0.8100,85100
2025-06-06,0.8206,0.8400,0.7900,0.8150,118400
2025-06-09,0.8048,0.8315,0.7933,0.8200,40000
2025-06-10,0.8000,0.8250,0.7611,0.8000,77600
2025-06-11,0.7900,0.7925,0.7640,0.7900,43500
